In [33]:
!pip3 install pyyaml
!pip3 install joblib
!pip3 install ensure


In [34]:
!pip3 install python-box

In [35]:
import yaml
import joblib
import ensure
# we have to work with in chicken-disease folder path:/Users/shivamsaini/chicken-disease


In [36]:
cd /Users/shivamsaini/chicken-disease


/Users/shivamsaini/chicken-disease


In [37]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_file: Path
    unzip_dir: Path

In [38]:
# file_dir = os.path.dirname('/Users/shivamsaini/chicken-disease/src/cnnClassifier/constants')
# sys.path.append(file_dir)
%pwd

'/Users/shivamsaini/chicken-disease'

In [39]:
!whereis python

python: /opt/anaconda3/envs/chicken/bin/python


In [40]:
import sys
import os
sys.path.append(os.path.abspath('src'))
%pwd

'/Users/shivamsaini/chicken-disease'

In [41]:
from cnnClassifier.constants import CONFIG_FILE_PATH , PARAM_FILE_PATH 
from cnnClassifier.utils.common import read_yaml , create_directories

In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

        print(self.config)
        print(self.params)


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_file = config.local_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

obj=ConfigurationManager()


      

[2024-08-04 22:29:31,455: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-04 22:29:31,456: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'data_ingestion', 'source_url': 'https://github.com/shivamsaini7227/DataWeNeed/blob/main/archive.zip', 'local_file': '/Users/shivamsaini/chicken-disease/artifacts/dataingestion', 'unzip_dir': 'artifacts/dataingestion'}}
{'key': 'value'}
[2024-08-04 22:29:31,457: INFO: common: created directory at: artifacts]


In [43]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [51]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
!whereis python
from box import Box
from box.exceptions import BoxKeyError


python: /opt/anaconda3/envs/chicken/bin/python


In [53]:
try:
   config= ConfigurationManager()
   data_ingestion_config = config.get_data_ingestion_config()
   data_ingestion= DataIngestion(config=data_ingestion_config)
   data_ingestion.download_file()
   data_ingestion.extract_zip_file() 

except Exception as e:
     raise e


[2024-08-06 00:58:09,705: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-06 00:58:09,711: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'data_ingestion', 'source_url': 'https://github.com/shivamsaini7227/DataWeNeed/raw/main/archive.zip', 'local_file': '/Users/shivamsaini/chicken-disease/artifacts/dataingestion/archive.zip', 'unzip_dir': 'artifacts/dataingestion'}, 'base_model': {'root_dir': 'artifacts/base_model', 'base_model_path': 'artifacts/base_model/model.h5', 'update_model_path': 'artifacts/base_model/updated_model.h5'}}
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
[2024-08-06 00:58:09,713: INFO: common: created directory at: artifacts]
[2024-08-06 00:58:09,714: INFO: common: created directory at: data_ingestion]
[2024-08-06 00:58:09,716: INFO: 4185630111: File 